In [9]:
import torch
import torch.nn as nn
import random
from datasets import load_dataset

# === Parameters ===
SEQ_LENGTH = 75
EMBED_DIM = 64
HIDDEN_DIM = 128
NUM_LAYERS = 1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === Load data to get vocabulary ===
print("Loading dataset and vocab...")
dataset = load_dataset('sander-wood/irishman', split='train')
texts = dataset['abc notation']
vocab = sorted(set(''.join(texts)))
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = {i: ch for ch, i in char2idx.items()}
VocabSize = len(vocab)

# === Define model ===
class SimpleRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(VocabSize, EMBED_DIM)
        self.lstm = nn.LSTM(EMBED_DIM, HIDDEN_DIM, NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM, VocabSize)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = out.reshape(-1, HIDDEN_DIM)
        logits = self.fc(out)
        return logits, hidden

# === Load model ===
model = SimpleRNN().to(DEVICE)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model.eval()
print("Model loaded successfully!")

# === Generate ABC tune ===
M_options = ["M:6/8", "M:4/4", "M:3/4"]
L_options = ["L:1/8", "L:1/16", "L:1/4"]
K_options = ["K:D", "K:G", "K:C"]

start = f"""X:1
T:Generated Tune
{random.choice(M_options)}
{random.choice(L_options)}
{random.choice(K_options)}
"""

seq = [char2idx.get(c, 0) for c in start]
hidden = None

for _ in range(800):  # generate 800 characters
    inp_seq = seq[-SEQ_LENGTH:]
    inp = torch.tensor(inp_seq).unsqueeze(0).to(DEVICE)
    logits, hidden = model(inp, hidden)
    probs = torch.softmax(logits[-1], dim=0)
    idx = torch.multinomial(probs, 1).item()
    seq.append(idx)

# === Decode and print ===
generated = ''.join(idx2char[i] for i in seq)
print("\n=== Generated ABC Notation ===\n")
print(generated)


Loading dataset and vocab...


C:\Users\geome\AppData\Local\Temp\ipykernel_1204\4220505250.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_locat

Model loaded successfully!

=== Generated ABC Notation ===

X:1
T:Generated Tune
M:4/4
L:1/4
K:C
 G2 Bc Bc | d2 d2 ag | g4 ce | g2 g2 ed | BG AF d2 | c3 A cd | e2 g2 f2 | e4 dc | 
 cB cd ed | c4 EF | G2 AB cA | G6 |[M:3/4] C4 G4- | A4- A4- |[M:2/2]) (A2 c2 | c4) c2 | 
 d4 (dc) | (G E) D4 | (C2 D4) E2 | f2 (e d) c2 | =B4S c2 (B A) | (B2 A2) (A B c2) | 
 d4 f2 (3(f2 e2) a4 | (a g) f2 | (g a) (b2-"^sos4itTtichal's4" c'6 gf | e4 f2 (ge) f4 d2 | f2 d2 e2 d cB c2 d2 z2 C6 c6 | 
 (ff)(fd) | (c2 B4 A2!p! AE) (E2 C6) E2 ||[M:2/4] G8 z2 |: C2E | A<c B<B,2 B,2 A,2 | C2C2 C2 A,>B, C>CA,C, C4 | a2>g2 a>ec>^f e2c>B | 
 C<A,A,>A, E2<A2 =G>AB<d c2Bc d2>c2 c2fe | e8 d8"_fina! efgeeee z4 (!p!!fedc=BAgfe2).d2c2c2 f6 A4 | 
 (fg) (g4{a} g4) Tg6 e4 (fg) !fermata!G8 ::[M:3/8][K:C] [DG]bag e2g2 | d4 dcBc | ded g4 | 
 GD[CB]7 | [GB]2[GB]2 c2e2 | eeef e2d2 | e2c2 A2GA | cde2- d4 | ge d2g ed[dg-] | [dg]2e2{f} g
